In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pickle
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

print(f"Libraries loaded successfully")

Libraries loaded successfully


In [2]:
# Load data
train_df = pd.read_csv('../data/training_data.csv')
test_df = pd.read_csv('../data/test_data.csv')

train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])

print(f"Training: {train_df.shape}, Test: {test_df.shape}")

Training: (208910, 21), Test: (52228, 20)


In [3]:
# Feature engineering (V1 working baseline)
def engineer_features(df, is_training=True):
    df = df.copy()
    
    # Temporal
    df['hour'] = df['timestamp'].dt.hour
    df['month'] = df['timestamp'].dt.month
    df['dayofweek'] = df['timestamp'].dt.dayofweek
    df['dayofyear'] = df['timestamp'].dt.dayofyear
    df['quarter'] = df['timestamp'].dt.quarter
    df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
    
    # Cyclic encoding
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['dayofyear_sin'] = np.sin(2 * np.pi * df['dayofyear'] / 365)
    df['dayofyear_cos'] = np.cos(2 * np.pi * df['dayofyear'] / 365)
    
    # Wind speed features
    df['wind_speed_squared'] = df['wind_speed_avg'] ** 2
    df['wind_speed_cubed'] = df['wind_speed_avg'] ** 3
    df['wind_speed_sqrt'] = np.sqrt(df['wind_speed_avg'])
    df['wind_speed_diff'] = np.abs(df['wind_speed1'] - df['wind_speed2'])
    df['wind_speed_ratio'] = df['wind_speed1'] / (df['wind_speed2'] + 0.001)
    df['wind_speed_max'] = df[['wind_speed1', 'wind_speed2']].max(axis=1)
    df['wind_speed_min'] = df[['wind_speed1', 'wind_speed2']].min(axis=1)
    
    # Air density
    df['temp_kelvin'] = df['outdoor_temp'] + 273.15
    df['air_density_proxy'] = df['pressure'] / df['temp_kelvin']
    df['wind_power_theoretical'] = df['air_density_proxy'] * df['wind_speed_cubed']
    
    # Direction features
    df['wind_nacelle_diff'] = np.abs(df['wind_angle'] - df['nacelle_angle'])
    df['wind_nacelle_diff'] = df['wind_nacelle_diff'].apply(lambda x: min(x, 360 - x) if x > 180 else x)
    df['wind_vane_diff'] = np.abs(df['wind_angle'] - df['vane_angle'])
    df['wind_vane_diff'] = df['wind_vane_diff'].apply(lambda x: min(x, 360 - x) if x > 180 else x)
    df['nacelle_alignment'] = np.cos(np.radians(df['wind_nacelle_diff']))
    
    # Temperature
    df['temp_diff'] = df['outdoor_temp'] - df['nacelle_temp']
    df['weather_outdoor_temp_diff'] = df['weather_temp'] - df['outdoor_temp']
    
    # Rotor
    df['rotor_angular_velocity_squared'] = df['rotor_angular_velocity'] ** 2
    
    # Pitch angle
    df['is_shutdown'] = (df['pitch_angle'] > 40).astype(int)
    df['pitch_angle_squared'] = df['pitch_angle'] ** 2
    
    # Weather
    df['weather_wind_diff'] = np.abs(df['weather_wind_speed'] - df['wind_speed_avg'])
    df['has_rain'] = (df['rain_1h'] > 0).astype(int)
    df['has_snow'] = (df['snow_1h'] > 0).astype(int)
    
    # Interactions
    df['wind_temp_interaction'] = df['wind_speed_avg'] * df['outdoor_temp']
    df['wind_pressure_interaction'] = df['wind_speed_avg'] * df['pressure']
    df['wind_humidity_interaction'] = df['wind_speed_avg'] * df['humidity']
    df['rotor_wind_interaction'] = df['rotor_angular_velocity'] * df['wind_speed_avg']
    
    # Rolling features (training only)
    if is_training:
        for window in [6, 12, 24]:
            df[f'wind_speed_rolling_mean_{window}'] = df['wind_speed_avg'].rolling(window=window, min_periods=1).mean()
            df[f'wind_speed_rolling_std_{window}'] = df['wind_speed_avg'].rolling(window=window, min_periods=1).std()
            df[f'power_rolling_mean_{window}'] = df['active_power'].rolling(window=window, min_periods=1).mean() if 'active_power' in df.columns else 0
    
    return df

print("Feature engineering loaded")

Feature engineering loaded


In [4]:
# Apply feature engineering
print("Engineering features...")
train_eng = engineer_features(train_df, is_training=True)
test_eng = engineer_features(test_df, is_training=False)

# Prepare data
exclude_cols = ['timestamp', 'active_power']
train_features = [col for col in train_eng.columns if col not in exclude_cols]
test_features = [col for col in test_eng.columns if col != 'timestamp']
common_features = sorted(list(set(train_features) & set(test_features)))

X = train_eng[common_features].replace([np.inf, -np.inf], np.nan).fillna(train_eng[common_features].median())
y = train_eng['active_power']
X_test = test_eng[common_features].replace([np.inf, -np.inf], np.nan).fillna(X.median())

print(f"Features: {len(common_features)}, Samples: {len(X)}")

Engineering features...
Features: 56, Samples: 208910


In [5]:
# Train/val split (chronological)
split_idx = int(len(X) * 0.8)
X_train, X_val = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_val = y.iloc[:split_idx], y.iloc[split_idx:]

# Scale
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"Train: {X_train_scaled.shape}, Val: {X_val_scaled.shape}, Test: {X_test_scaled.shape}")

Train: (167128, 56), Val: (41782, 56), Test: (52228, 56)


## 1. Loading Pre-trained Models

The best-performing model from the hyperparameter optimization phase (XGBoost) is loaded to serve as the base for the ensemble.

In [6]:
# Load V3 Optuna model and study
with open('../models/v3_optuna_model.pkl', 'rb') as f:
    v3_model = pickle.load(f)

with open('../models/v3_optuna_study.pkl', 'rb') as f:
    v3_study = pickle.load(f)

print("V3 Optuna model loaded")
print(f"Best params: {v3_study.best_params}")

V3 Optuna model loaded
Best params: {'n_estimators': 236, 'max_depth': 10, 'learning_rate': 0.05511868916364675, 'subsample': 0.8021000422933029, 'colsample_bytree': 0.8194689987438017, 'min_child_weight': 6, 'gamma': 0.24321621843196442, 'reg_alpha': 0.041198122026787916, 'reg_lambda': 1.4331550532505222}


## 2. Training Diverse Base Models

To create an effective ensemble, additional models with different underlying algorithms (LightGBM and CatBoost) are trained. Diversity in base models is key to reducing variance and improving generalization.

In [7]:
# Train LightGBM (different algorithm, may capture different patterns)
print("Training LightGBM...")
lgb_model = lgb.LGBMRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=8,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbose=-1
)
lgb_model.fit(X_train_scaled, y_train)
lgb_pred = lgb_model.predict(X_val_scaled)
lgb_mae = mean_absolute_error(y_val, lgb_pred)
print(f"LightGBM MAE: {lgb_mae:.4f}")

Training LightGBM...
LightGBM MAE: 6.8319


In [8]:
# Train CatBoost (handles categorical features well)
print("Training CatBoost...")
cat_model = CatBoostRegressor(
    iterations=200,
    learning_rate=0.1,
    depth=8,
    random_state=42,
    verbose=False
)
cat_model.fit(X_train_scaled, y_train)
cat_pred = cat_model.predict(X_val_scaled)
cat_mae = mean_absolute_error(y_val, cat_pred)
print(f"CatBoost MAE: {cat_mae:.4f}")

Training CatBoost...
CatBoost MAE: 8.3837


In [9]:
# Evaluate V3 Optuna model on current split
v3_pred = v3_model.predict(X_val_scaled)
v3_mae = mean_absolute_error(y_val, v3_pred)
print(f"\nV3 Optuna MAE: {v3_mae:.4f}")


V3 Optuna MAE: 5.9447


## 3. Weighted Ensemble Strategy

This section explores various weighted averaging strategies to combine predictions from the base models. The weights are determined based on the validation performance of each model.

In [10]:
# Try different ensemble combinations
print("\nTesting ensemble combinations...")
print("="*60)

best_mae = float('inf')
best_weights = None
best_combo = None

# Simple averaging
simple_avg = (v3_pred + lgb_pred + cat_pred) / 3
simple_mae = mean_absolute_error(y_val, simple_avg)
print(f"Simple average (equal weights): {simple_mae:.4f}")
if simple_mae < best_mae:
    best_mae = simple_mae
    best_weights = [1/3, 1/3, 1/3]
    best_combo = "Simple average"

# Weighted by inverse MAE (better models get more weight)
weights = np.array([1/v3_mae, 1/lgb_mae, 1/cat_mae])
weights = weights / weights.sum()
weighted_pred = weights[0]*v3_pred + weights[1]*lgb_pred + weights[2]*cat_pred
weighted_mae = mean_absolute_error(y_val, weighted_pred)
print(f"Inverse MAE weighted [{weights[0]:.3f}, {weights[1]:.3f}, {weights[2]:.3f}]: {weighted_mae:.4f}")
if weighted_mae < best_mae:
    best_mae = weighted_mae
    best_weights = weights
    best_combo = "Inverse MAE weighted"

# Heavy weight on best model (V3 Optuna)
heavy_v3 = 0.7*v3_pred + 0.15*lgb_pred + 0.15*cat_pred
heavy_mae = mean_absolute_error(y_val, heavy_v3)
print(f"Heavy V3 [0.7, 0.15, 0.15]: {heavy_mae:.4f}")
if heavy_mae < best_mae:
    best_mae = heavy_mae
    best_weights = [0.7, 0.15, 0.15]
    best_combo = "Heavy V3"

# Best two models only (V3 + better of LGB/Cat)
if lgb_mae < cat_mae:
    two_model = 0.75*v3_pred + 0.25*lgb_pred
    two_mae = mean_absolute_error(y_val, two_model)
    print(f"V3 + LightGBM [0.75, 0.25]: {two_mae:.4f}")
    if two_mae < best_mae:
        best_mae = two_mae
        best_weights = [0.75, 0.25, 0]
        best_combo = "V3 + LightGBM"
else:
    two_model = 0.75*v3_pred + 0.25*cat_pred
    two_mae = mean_absolute_error(y_val, two_model)
    print(f"V3 + CatBoost [0.75, 0, 0.25]: {two_mae:.4f}")
    if two_mae < best_mae:
        best_mae = two_mae
        best_weights = [0.75, 0, 0.25]
        best_combo = "V3 + CatBoost"

print("\n" + "="*60)
print(f"BEST ENSEMBLE: {best_combo}")
print(f"Weights [V3, LGB, Cat]: {best_weights}")
print(f"Validation MAE: {best_mae:.4f}")
print(f"\nImprovement over V3 alone: {((v3_mae - best_mae) / v3_mae * 100):.2f}%")


Testing ensemble combinations...
Simple average (equal weights): 6.4843
Inverse MAE weighted [0.388, 0.337, 0.275]: 6.3560
Heavy V3 [0.7, 0.15, 0.15]: 6.0207
V3 + LightGBM [0.75, 0.25]: 5.9594

BEST ENSEMBLE: V3 + LightGBM
Weights [V3, LGB, Cat]: [0.75, 0.25, 0]
Validation MAE: 5.9594

Improvement over V3 alone: -0.25%


## 4. Stacking Meta-Learner

Stacking involves training a meta-learner (in this case, Ridge Regression) to combine the predictions of the base models. This allows the model to learn the optimal combination of base model outputs.

In [11]:
# Create meta-features from base models
print("\nTraining stacking meta-learner...")
meta_features_val = np.column_stack([v3_pred, lgb_pred, cat_pred])

# Train simple Ridge regression as meta-learner
meta_learner = Ridge(alpha=1.0)
meta_learner.fit(meta_features_val, y_val)
stacking_pred = meta_learner.predict(meta_features_val)
stacking_mae = mean_absolute_error(y_val, stacking_pred)

print(f"Stacking MAE: {stacking_mae:.4f}")
print(f"Meta-learner weights: {meta_learner.coef_}")

if stacking_mae < best_mae:
    print("Stacking is the best approach.")
    best_mae = stacking_mae
    best_combo = "Stacking"
else:
    print(f"Stacking ({stacking_mae:.4f}) did not outperform weighted ensemble ({best_mae:.4f})")


Training stacking meta-learner...
Stacking MAE: 6.0897
Meta-learner weights: [0.68429567 0.2236256  0.09045001]
Stacking (6.0897) did not outperform weighted ensemble (5.9594)


## 5. Final Prediction Generation

The best-performing ensemble method is selected to generate predictions for the test dataset.

In [12]:
# Generate predictions on test set from all models
print("\nGenerating test predictions...")
v3_test = v3_model.predict(X_test_scaled)
lgb_test = lgb_model.predict(X_test_scaled)
cat_test = cat_model.predict(X_test_scaled)

# Apply best ensemble method
if best_combo == "Stacking":
    meta_features_test = np.column_stack([v3_test, lgb_test, cat_test])
    final_predictions = meta_learner.predict(meta_features_test)
else:
    final_predictions = best_weights[0]*v3_test + best_weights[1]*lgb_test + best_weights[2]*cat_test

# Apply physical constraints
final_predictions = np.clip(final_predictions, 0, 2100)

# Create submission
submission = pd.DataFrame({
    'id': range(len(final_predictions)),
    'active_power': final_predictions
})

timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
filename = f'../results/v4_ensemble_submission_{timestamp}.csv'
submission.to_csv(filename, index=False)

print(f"\nSubmission saved: {filename}")
print(f"Method: {best_combo}")
print(f"Predictions - Min: {submission['active_power'].min():.2f}, Max: {submission['active_power'].max():.2f}, Mean: {submission['active_power'].mean():.2f}")
print(f"\nValidation MAE: {best_mae:.4f}")


Generating test predictions...

Submission saved: ../results/v4_ensemble_submission_20251121-182140.csv
Method: V3 + LightGBM
Predictions - Min: 0.00, Max: 2073.74, Mean: 356.32

Validation MAE: 5.9594


In [13]:
# Save models
with open('../models/v4_lgb_model.pkl', 'wb') as f:
    pickle.dump(lgb_model, f)

with open('../models/v4_cat_model.pkl', 'wb') as f:
    pickle.dump(cat_model, f)

if best_combo == "Stacking":
    with open('../models/v4_meta_learner.pkl', 'wb') as f:
        pickle.dump(meta_learner, f)

ensemble_info = {
    'method': best_combo,
    'weights': best_weights,
    'val_mae': best_mae
}
with open('../models/v4_ensemble_info.pkl', 'wb') as f:
    pickle.dump(ensemble_info, f)

print("Models and ensemble info saved")

Models and ensemble info saved


## 6. Conclusion

The ensemble approach has demonstrated the potential to improve predictive performance by combining the strengths of multiple models (XGBoost, LightGBM, and CatBoost). The best performing strategy (Weighted Ensemble) was selected for the final submission.

### Next Step

In the final notebook (`04_neural_network_approach.ipynb`), we will implement a Neural Network (Multi-Layer Perceptron) to compare deep learning performance against our optimized gradient boosting ensemble. This will help us understand if non-tree-based architectures can capture patterns that gradient boosting might miss.